In [11]:
import os
import sys
import time
import pandas as pd
import time
import sys
from nltk import sent_tokenize
import logging
#print only errors
import tqdm
from multiprocessing import Pool, freeze_support
import json
sys.path.append("./..")
from postprocessing.rouge import get_rouge_score
import pickle
import numpy as np
from matplotlib import pyplot as plt
import plotly.figure_factory as ffdemo
import pickle
import random

In [2]:




beam_files = ["again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_1", \
              "again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_2", \
              "again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_3", \
              "again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_4", \
              "again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_5", \
              "again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_6"]
           


mixtures = ["head_0", "head_1"]
def extract_source_reference_and_prediction(file):
    article = []
    reference = []
    prediction = []
    with open(file, "r") as f:
        lines = f.readlines()
        cur_index = 0
        for index , line in enumerate(lines):
            if index % 4 == 0:
                article.append(line)
            elif index % 4 == 1:
                reference.append(line)
            elif index % 4 == 2:
                prediction.append(line)
            elif index % 4 == 3:
                cur_index += 1
    return article, reference, prediction

def create_all_possible_combinations(res):
    # indexed by beam size, mixture type and article index in the dataset
    keys = []
    candidates = []
    references = []
    articles = []
    for mixture_type in tqdm.tqdm(res.keys()):
        if mixture_type in ["head_0", "head_1"]:
            for index in res[mixture_type].keys():
                article = res[mixture_type][index]["article"]
                reference = res[mixture_type][index]["reference"]
                for beam_size in res[mixture_type][index]["predictions"].keys():
                    key = (beam_size, mixture_type, index)
                    keys.append(key)
                    candidates.append(res[mixture_type][str(index)]["predictions"][str(beam_size)])
                    references.append(reference)
                    articles.append(article)
    print(len(keys))
    print(len(candidates))
    print(len(references))
    print(len(articles))

    return keys, candidates, references, articles

In [8]:
results = {}
for mixture in mixtures:
    results[mixture] = {}
    print("mixture : {0}".format(mixture))
    for index,beam_file in tqdm.tqdm(enumerate(beam_files)):
        print(index, beam_file)
        mixture_folder = os.path.join(beam_file, mixture)
        relevant_filepath = os.path.join(mixture_folder, "test_outfinal.txt")
        article, reference, prediction = extract_source_reference_and_prediction(relevant_filepath)

        if index == 0:
            for idx, line in enumerate(article):
                results[mixture][str(idx)] = {"article": line, "reference": reference[idx], "predictions" : {}}
                results[mixture][str(idx)]["predictions"][str(index + 1)] = prediction[idx]
        else:
            for idx, line in enumerate(article):
                results[mixture][str(idx)]["predictions"][str(index + 1)] = prediction[idx]
            #sort the result[mixture] by index 
            results[mixture] = dict(sorted(results[mixture].items(), key=lambda x: int(x[0])))
#save the json file
with open("train_data_for_beam_size_prediction.json", "w") as f:
    json.dump(results, f)

mixture : head_0


1it [00:00,  8.87it/s]

0 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_1
1 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_2


4it [00:00, 11.95it/s]

2 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_3
3 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_4
4 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_5


...

1 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_2


3it [00:00,  9.73it/s]

2 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_3
3 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_4
4 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_5


6it [00:00, 11.48it/s]


5 again_tests//overlap_supervision_div_loss_1_pre_lm_loss_two_head_grad_acc_10_lr_1e4_epoch_1_train_cnn_num_beam_6


In [3]:
#
unprocessed_beam_results_file = "beam_rouge_result_unprocessed.pkl"
if os.path.exists(unprocessed_beam_results_file):
    with open(unprocessed_beam_results_file, "rb") as f:
        unprocessed_beam_results = pickle.load(f)
print(len(unprocessed_beam_results.keys()))

#read the json file
with open("train_data_for_beam_size_prediction.json", "r") as f:
    results = json.load(f)

#now combine the two 


120000


In [4]:
for key in tqdm.tqdm(unprocessed_beam_results.keys()):
    beam_size, mixture_type, index = key
    if "rouge_scores" not in results[mixture_type][str(index)].keys():
        results[mixture_type][str(index)]['rouge_scores'] = {str(beam_size) : unprocessed_beam_results[key]}
    else:
        results[mixture_type][str(index)]['rouge_scores'][str(beam_size)] = unprocessed_beam_results[key]



100%|██████████| 120000/120000 [00:00<00:00, 565660.15it/s]


In [4]:
results['head_0']['0']['rouge_scores']

{'1': {'rouge_1_f_score': 0.30769,
  'rouge_2_f_score': 0.22222,
  'rouge_l_f_score': 0.18462},
 '2': {'rouge_1_f_score': 0.30303,
  'rouge_2_f_score': 0.21875,
  'rouge_l_f_score': 0.18182},
 '3': {'rouge_1_f_score': 0.30303,
  'rouge_2_f_score': 0.21875,
  'rouge_l_f_score': 0.18182},
 '4': {'rouge_1_f_score': 0.29851,
  'rouge_2_f_score': 0.21538,
  'rouge_l_f_score': 0.17911},
 '5': {'rouge_1_f_score': 0.29851,
  'rouge_2_f_score': 0.21538,
  'rouge_l_f_score': 0.17911},
 '6': {'rouge_1_f_score': 0.29851,
  'rouge_2_f_score': 0.21538,
  'rouge_l_f_score': 0.17911}}

In [13]:
#for head_0
best = []
worst = []
beam_1 = []
beam_2 = []
beam_3 = []
beam_4 = []
beam_5 = []
beam_6 = []
for index in results['head_0'].keys():
    scores = results['head_0'][index]['rouge_scores']
    rouge_1_scores = [(int(k),x['rouge_1_f_score']) for k,x in scores.items()]
    #sort by the rouge-1 score in descending order
    beam_1.append(scores['1']['rouge_1_f_score'])
    beam_2.append(scores['2']['rouge_1_f_score'])
    beam_3.append(scores['3']['rouge_1_f_score'])
    beam_4.append(scores['4']['rouge_1_f_score'])
    beam_5.append(scores['5']['rouge_1_f_score'])
    beam_6.append(scores['6']['rouge_1_f_score'])
    
    rouge_1_scores = sorted(rouge_1_scores, key=lambda x: x[1], reverse=True)
    best_index = rouge_1_scores[0][0]
    best_score = rouge_1_scores[0][1]
    worst_score = rouge_1_scores[-1][1]

    best.append((best_score))
    worst.append((worst_score))
    results['head_0'][index]['best_rouge'] = best_index

                      


In [14]:
frequency = [0 for i in range(0,7)]
for index in results['head_0'].keys():
    best_index = results['head_0'][index]['best_rouge']
    frequency[best_index] += 1
print(frequency)


[0, 4447, 3248, 1074, 595, 383, 253]


: 

In [6]:
#for head_1
best = []
worst = []
beam_1 = []
beam_2 = []
beam_3 = []
beam_4 = []
beam_5 = []
beam_6 = []
for index in results['head_1'].keys():
    scores = results['head_1'][index]['rouge_scores']
    rouge_1_scores = [(int(k),x['rouge_1_f_score']) for k,x in scores.items()]
    #sort by the rouge-1 score in descending order
    beam_1.append(scores['1']['rouge_1_f_score'])
    beam_2.append(scores['2']['rouge_1_f_score'])
    beam_3.append(scores['3']['rouge_1_f_score'])
    beam_4.append(scores['4']['rouge_1_f_score'])
    beam_5.append(scores['5']['rouge_1_f_score'])
    beam_6.append(scores['6']['rouge_1_f_score'])
    
    rouge_1_scores = sorted(rouge_1_scores, key=lambda x: x[1], reverse=True)
    best_index = rouge_1_scores[0][0]
    best_score = rouge_1_scores[0][1]
    worst_score = rouge_1_scores[-1][1]

    best.append((best_score))
    worst.append((worst_score))
    results['head_1'][index]['best_rouge'] = best_index

In [8]:
results['head_1']['0']

{'article': "Editor 's note : In our Behind the Scenes series , CNN correspondents share their experiences in covering news and analyze the stories behind the events . Here , Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill . An inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in Miami before trial . MIAMI , Florida -LRB- CNN -RRB- -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the `` forgotten floor . '' Here , inmates with the most severe mental illnesses are incarcerated until they 're ready to appear in court . Most often , they face drug charges or charges of assaulting an officer -- charges that Judge Steven Leifman says are usually `` avoidable felonies . '' He says the arrests often result from confrontations with police . Mentally ill people often wo n't do what they 're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become mo

In [21]:
print(np.average(best))
print(np.average(worst))
print(np.average(beam_1))
print(np.average(beam_2))
print(np.average(beam_3))
print(np.average(beam_4))
print(np.average(beam_5))
print(np.average(beam_6))

0.426626264
0.26948863199999995
0.33827261799999997
0.351188635
0.352801064
0.352877055
0.35148168
0.352945295


In [7]:
diff = [best[i] - worst[i] for i in range(len(best))]
print(np.average(diff))



0.15713763199999997


In [10]:
save_file = "train_data_for_beam_size_prediction_with_rouge_scores.json"
#save as a pickle file
with open(save_file, "wb") as f:
    pickle.dump(results, f)

In [12]:
#split head 0 and head 1 into 80/20 for training and testing
train_head_0_filepath = "./train_head_0.pkl"
train_head_1_filepath = "./train_head_1.pkl"

valid_head_0_filepath = "./valid_head_0.pkl"
valid_head_1_filepath = "./valid_head_1.pkl"

#head 0
head_0 = results['head_0']
head_0_keys = list(head_0.keys())

#shuffle the keys
random.shuffle(head_0_keys)

#split into 80/20
train_head_0_data = {k:head_0[k] for k in head_0_keys[:int(0.8*len(head_0_keys))]}
valid_head_0_data = {k:head_0[k] for k in head_0_keys[int(0.8*len(head_0_keys)):]}

print(len(train_head_0_data.keys()))
print(len(valid_head_0_data.keys()))

#now head 1
head_1 = results['head_1']
head_1_keys = list(head_1.keys())

#shuffle the keys
random.shuffle(head_1_keys)

#split into 80/20
train_head_1_data = {k:head_1[k] for k in head_1_keys[:int(0.8*len(head_1_keys))]}
valid_head_1_data = {k:head_1[k] for k in head_1_keys[int(0.8*len(head_1_keys)):]}

print(len(train_head_1_data.keys()))
print(len(valid_head_1_data.keys()))

#save all the files as json 
with open(train_head_0_filepath, "wb") as f:
    pickle.dump(train_head_0_data, f)
with open(train_head_1_filepath, "wb") as f:
    pickle.dump(train_head_1_data, f)
with open(valid_head_0_filepath, "wb") as f:
    pickle.dump(valid_head_0_data, f)
with open(valid_head_1_filepath, "wb") as f:
    pickle.dump(valid_head_1_data, f)



8000
2000
8000
2000
